In [ ]:
# ------------------------------------------
# VISA Communication Setup using PyVISA
# ------------------------------------------

import pyvisa  # PyVISA is used to talk to instruments like oscilloscopes, multimeters, etc.

print("--- Setting Up pyvisa ---")

try:
    # Create a VISA Resource Manager (used to list and connect to instruments)
    # By default, PyVISA uses National Instruments (NI-VISA) backend.
    # '@py' forces the use of PyVISA-py (pure Python backend, no need for NI software)

    rm = pyvisa.ResourceManager('@py')  # '@py' is best for testing without actual hardware
    print("✅ PyVISA Resource Manager created successfully.")
    print(f"🔌 Backend in use: {rm.backend}")

except Exception as e:
    print("❌ Error while creating Resource Manager:")
    print(f"   → {e}")
    print("💡 Tip: Make sure 'pyvisa' is installed correctly.")
    print("         If you're using hardware with NI-VISA, ensure NI-VISA drivers are also installed.")
    print("📘 Fallback: Continuing with conceptual examples only.")

    rm = None  # If setup fails, set rm to None to avoid crashes in further code

In [17]:
print("\n--- Listing Available VISA Resources ---")

import pyvisa
rm = pyvisa.ResourceManager('@py')  # Or without '@py' if using NI-VISA

# Make sure the resource manager (rm) is available
if rm:
    try:
        # Try to list all connected VISA instruments
        resources = rm.list_resources()
        print("📡 Found the following VISA resources:")
        
        # If resources are found, list them
        if resources:
            for res in resources:
                print(f"  • {res}")
        else:
            # No instruments found (normal if none are connected)
            print("⚠️ No VISA instruments found.")
            print("ℹ️  This is normal if no physical instruments are connected.")
            print("💡 Examples may use dummy or simulated addresses like TCPIP0::...")

    except pyvisa.errors.VisaIOError as e:
        # Catch any error from VISA communication
        print(f"❌ Error listing VISA resources: {e}")
        print("🛠️  This may happen if the VISA backend is not installed or configured correctly.")

else:
    # If resource manager creation failed earlier
    print("⚠️ Resource Manager not available. Skipping resource listing.")


--- Listing Available VISA Resources ---
📡 Found the following VISA resources:
  • ASRL/dev/cu.debug-console::INSTR
  • ASRL/dev/cu.Bluetooth-Incoming-Port::INSTR


In [18]:
# File: open_visa_instrument.py

import pyvisa  # Import the PyVISA library

print("\n--- Opening a Connection to an Instrument ---")

# Step 1: Create a Resource Manager using PyVISA-py backend
try:
    rm = pyvisa.ResourceManager('@py')  # '@py' ensures using the pure Python backend (no NI-VISA needed)
    print("✅ PyVISA Resource Manager created successfully.")
    print(f"Backend: {rm.backend}")
except Exception as e:
    print(f"❌ Failed to create Resource Manager: {e}")
    rm = None

# Step 2: Define a dummy VISA address (or use one from rm.list_resources())
dummy_instrument_address = 'ASRL1::INSTR'  # Example: Serial port or test resource

# Step 3: Try to open a connection to the resource
if rm:
    try:
        # 'with' block automatically closes the connection when done
        with rm.open_resource(dummy_instrument_address) as my_instrument:
            print(f"\n🔗 Successfully connected to: {dummy_instrument_address}")
            print(f"Interface Type: {my_instrument.interface_type}")  # e.g., 'ASRL', 'GPIB', 'USB'
            print(f"Resource Name: {my_instrument.resource_name}")
            print("📡 Connection is active inside this 'with' block.")
            # You could send SCPI commands here, e.g., my_instrument.query("*IDN?")

        # Outside the 'with' block, the connection is automatically closed
        print(f"🔒 Connection to {dummy_instrument_address} is now closed.")

    except pyvisa.errors.VisaIOError as e:
        print(f"\n⚠️ Could not open resource {dummy_instrument_address}: {e}")
        print("ℹ️ This might happen if no instrument is connected or address is incorrect.")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
else:
    print("❌ Resource Manager not available. Cannot open instrument.")


--- Opening a Connection to an Instrument ---
✅ PyVISA Resource Manager created successfully.
❌ Failed to create Resource Manager: 'ResourceManager' object has no attribute 'backend'
❌ Resource Manager not available. Cannot open instrument.


In [20]:
import pyvisa

print("\n--- SCPI Commands: Writing and Reading Data ---")

# Step 1: Create a PyVISA Resource Manager (if not already created)
try:
    rm = pyvisa.ResourceManager('@py')  # '@py' uses the PyVISA-py backend
    print("✅ PyVISA Resource Manager created.")
except Exception as e:
    print(f"❌ Failed to create Resource Manager: {e}")
    rm = None

# Step 2: Interact with a simulated SCPI instrument
if rm:
    try:
        # Simulated instrument address (works only if the simulator backend supports it)
        simulated_instrument_address = 'TCPIP::127.0.0.1::INSTR'  # You can also try 'TCPIP::127.0.0.1::INSTR'

        with rm.open_resource(simulated_instrument_address) as inst:
            print(f"\n🔗 Connected to simulated instrument: {inst.resource_name}")

            # --- 1. Query identity ---
            idn_response = inst.query('*IDN?')  # Standard SCPI command for instrument identity
            print(f"🆔 Instrument ID: {idn_response.strip()}")

            # --- 2. Set a frequency using a sine wave command ---
            frequency_to_set = 1000  # Hz
            inst.write(f'APPL:SIN {frequency_to_set}')  # SCPI command to apply sine waveform
            print(f"📡 Sent command: APPL:SIN {frequency_to_set}")

            # --- 3. Query the current frequency ---
            freq_response = inst.query('APPL:SIN?')
            print(f"🔍 Queried Frequency: {freq_response.strip()} Hz")

            # --- 4. Measure a voltage value ---
            voltage_response = inst.query('MEAS:VOLT:DC?')  # SCPI command to measure DC voltage
            print(f"🔋 Measured DC Voltage: {voltage_response.strip()} V")

            # --- 5. Send multiple SCPI commands in one line ---
            inst.write('VOLT 2.5; OUTP ON')  # Set voltage and enable output
            print("⚙️ Sent: VOLT 2.5; OUTP ON")

            # --- 6. Query output state after write ---
            inst.write('OUTP?')       # Ask if output is ON or OFF
            output_state = inst.read()  # Read the result
            print(f"✅ Output State: {output_state.strip()}")

    except pyvisa.errors.VisaIOError as e:
        print(f"\n⚠️ VISA I/O Error: {e}")
        print("💡 This is common if no simulation backend is active or address is invalid.")
    except Exception as e:
        print(f"\n❌ Unexpected error: {e}")
else:
    print("❌ No Resource Manager available. Cannot proceed.")


--- SCPI Commands: Writing and Reading Data ---
✅ PyVISA Resource Manager created.

❌ Unexpected error: [Errno 32] Broken pipe


In [ ]:
import pyvisa

rm = pyvisa.ResourceManager('@py')  # Force using PyVISA-py backend

# Open the socket resource
inst = rm.open_resource("TCPIP0::192.168.0.103::5025::SOCKET")

# ✅ Set termination settings
inst.read_termination = '\n'
inst.write_termination = '\n'
inst.timeout = 3000  # in milliseconds (3 seconds, adjust if needed)

# Now query
print(inst.query("*IDN?"))       # Should print: RPi,SCPI-VISASIM,001,1.0
print(inst.query("MEAS:TEMP?"))  # Should print: 28.5

In [11]:
import socket

# --- Configuration ---
HOST = '192.168.0.103'   # Replace with your Raspberry Pi's IP address
PORT = 5025              # The port your server is listening on

print("💡 LED Toggle Client Starting...")

# Function to send a command and receive a response
def send_scpi_command(command):
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((HOST, PORT))
            print(f"🔌 Connected to {HOST}:{PORT}")

            # Send command
            s.sendall((command + '\n').encode('utf-8'))
            print(f"📤 Sent: {command}")

            # Receive response
            data = s.recv(1024).decode('utf-8').strip()
            print(f"📥 Response: {data}")
            return data

    except ConnectionRefusedError:
        print("❌ Could not connect to server. Is it running?")
    except Exception as e:
        print(f"⚠️ Error: {e}")

# --- Example Usage ---
if __name__ == '__main__':
    # Send commands IDN?
    send_scpi_command("*IDN?")
    
    # Send commands MEAS:TEMP?
    send_scpi_command("MEAS:TEMP?")

    # Toggle LED ON
    send_scpi_command("LED ON")

    # Wait a moment (optional)
    import time
    time.sleep(2)

    # Toggle LED OFF
    send_scpi_command("LED OFF")

    # Optionally query the state
    send_scpi_command("LED?")

💡 LED Toggle Client Starting...
🔌 Connected to 192.168.0.103:5025
📤 Sent: *IDN?
📥 Response: RPi,SCPI-GPIO,001,1.0
🔌 Connected to 192.168.0.103:5025
📤 Sent: MEAS:TEMP?
📥 Response: 54.50
🔌 Connected to 192.168.0.103:5025
📤 Sent: LED ON
📥 Response: OK:LED_ON
🔌 Connected to 192.168.0.103:5025
📤 Sent: LED OFF
📥 Response: OK:LED_OFF
🔌 Connected to 192.168.0.103:5025
📤 Sent: LED?
📥 Response: ERR:UNKNOWN_COMMAND


In [21]:
import socket
import time
import random

# Raspberry Pi SCPI Client for Voltage and Resistance simulation

def query_rpi_scpi(command, host="192.168.0.103", port=5025):
    """
    Sends a SCPI command to the Raspberry Pi SCPI server and returns the response.
    """
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
        client_socket.connect((host, port))
        client_socket.sendall((command + '\n').encode('utf-8'))
        response = client_socket.recv(1024).decode('utf-8').strip()
        return response

# Simulate multiple voltage readings
print("\n--- Simulating DC Voltage Measurements from Raspberry Pi ---")

try:
    # Identify the instrument
    idn = query_rpi_scpi("*IDN?")
    print(f"ID: {idn}")

    # Simulate configuration step (for conceptual flow)
    print("Configured Raspberry Pi for DC Voltage measurement.")

    # Take multiple readings (simulate noise in values)
    num_readings = 5
    readings = []
    for i in range(num_readings):
        temp_reading = query_rpi_scpi("MEAS:TEMP?")
        try:
            temp_val = float(temp_reading) + random.uniform(-0.01, 0.01)  # add noise
            readings.append(temp_val)
            print(f"  Reading {i+1}: {temp_val:.2f} °C")
        except ValueError:
            print(f"  Reading {i+1}: Invalid value received: '{temp_reading}'")
        time.sleep(0.5)

    # Display summary
    if readings:
        average = sum(readings) / len(readings)
        print(f"\nAverage Temperature: {average:.2f} °C")

    # Simulate resistance measurement
    print("\nNow simulating a resistance measurement...")
    resistance_sim = round(random.uniform(500.0, 600.0), 2)
    print(f"Measured Resistance (simulated): {resistance_sim} Ohms")

except Exception as e:
    print(f"Error: {e}")


--- Simulating DC Voltage Measurements from Raspberry Pi ---
ID: RPi,SCPI-GPIOSIM,001,1.0
Configured Raspberry Pi for DC Voltage measurement.
  Reading 1: 54.99 °C
  Reading 2: 53.49 °C
  Reading 3: 54.49 °C
  Reading 4: 55.00 °C
  Reading 5: 54.49 °C

Average Temperature: 54.49 °C

Now simulating a resistance measurement...
Measured Resistance (simulated): 513.21 Ohms
